In [ ]:
import os
import numpy as np
import pandas as pd
import csv
import re

## extract answer

In [ ]:
def extract_answer(text):
    """Extract answer letter A-D from raw_response."""
    cleaned = re.sub(r'[\*\_\"\']', '', text.strip().lower())
    patterns = [
        r'answer\s*[:\-]?\s*\(?([a-d])\)?',
        r'the answer is\s*\(?([a-d])\)?',
        r'the correct answer is\s*\(?([a-d])\)?',
        r'\b([a-d])\b\s+is the answer',
        r'\(?(answer)\)?\s*[:\-]?\s*\(?([a-d])\)?',
        r'\b([a-d])\b\.?$'
    ]
    for p in patterns:
        match = re.search(p, cleaned, re.IGNORECASE)
        if match:
            return match.group(2).upper() if match.lastindex >= 2 else match.group(1).upper()
    return "?"

## analyze results

In [ ]:
def analyze_results(cot, nocot):
    df=cot.merge(nocot[['index', 'raw_response']], on='index', how='inner')
    merged=df.merge(prompt_df[['index', 'valid_options']], on='index', how='inner')
    merged['predicted_cot']=merged['raw_response_x'].apply(lambda x: extract_answer(x))
    merged['predicted_nocot']=merged['raw_response_y'].apply(lambda x: extract_answer(x))
    print('acc for cot:', np.sum(merged['answer']==merged['predicted_cot'])/50)
    print('acc for nocot:', np.sum(merged['answer']==merged['predicted_nocot'])/50)
    ## extract cases when no_cot predictions are true but cot are wrong
    strange_cases=merged[(merged['answer']==merged['predicted_nocot'])&(merged['answer']!=merged['predicted_cot'])]
    ## extract cases when adding cot are correct and no cot are wrong
    good_cases=merged[(merged['answer']==merged['predicted_cot'])&(merged['answer']!=merged['predicted_nocot'])]
    return strange_cases, good_cases
    

In [ ]:
prompt_df=pd.read_csv('Data/prompt_df.csv')
cot=pd.read_csv('openai_o4_mini_cot.csv')
nocot=pd.read_csv('o4_mini_no_cot.csv')
strange_cases, good_cases =analyze_results(cot,nocot)